In [1]:
import pandas as pd
import pickle
import numpy as np
from transformers import AutoTokenizer
import itertools

In [2]:
tokenizer = AutoTokenizer.from_pretrained("labse_v4_good/tokenizer")

In [3]:
import re

def convert_ids_to_entities(ids, tokens=None, o_id=0, b_id=1, i_id=2):
    entities = []
    entity = []
    is_entity_started = False
    for index, token in enumerate(ids):
        if not is_entity_started and token == b_id:
            is_entity_started = True
            entity.append(index)
        elif is_entity_started and token == i_id:
            entity.append(index)
    
        if (is_entity_started and token == o_id) or (index == (len(ids) - 1)):
            is_entity_started = False
            entities.append(entity)
            entity = []
                
    if tokens is not None:
        tokens = np.array(tokens)
        token_entities = []
        for entity in entities:
            entity = np.array(entity)
            if len(entity) > 0:
                token_entity = tokens[entity]
                token_entities.append(token_entity.tolist())
            
        return token_entities
            
    return entities

def probs2str(classes, texts): # (4501, 70, 3)
    predictions = []
    input_ids_list = [tokenizer(text, 
                           add_special_tokens=True,
                           max_length=66,
                           padding="max_length",
                           return_offsets_mapping=False)["input_ids"] for text in texts]

    for clas, input_ids in zip(classes, input_ids_list):
        entities = convert_ids_to_entities(clas, input_ids)

        entities = list(itertools.chain(*entities))
        # entities = tokenizer.convert_ids_to_tokens(entities)
        # entities = [entity for entity in entities if (entity != "[PAD]") and (entity != "[SEP]")]
        # prediction = tokenizer.convert_tokens_to_string(entities)
        prediction = tokenizer.decode(entities, skip_special_tokens=True)
        prediction = re.sub(" - ", "-", prediction)
        prediction = re.sub(" & ", "&", prediction)
        prediction = re.sub("##", "", prediction)
        predictions.append(prediction)
    
    return predictions

In [4]:
with open("labse_v4_good/oof_df.pkl", "rb") as file:
    oof = pickle.load(file)

In [5]:
oof

,name,good,good_pos,strat,fold,0,1,2,3,4,...,56,57,58,59,60,61,62,63,64,65
0,petmax бантик леопард с красн розой 2шт,бантик,"(7, 13)",0,0,0,0,0,1,2,...,0,0,0,0,0,0,0,0,0,0
1,87191 бусы для елки шарики_87191,бусы,"(6, 10)",0,0,0,0,0,1,2,...,0,0,0,0,0,0,0,0,0,0
2,футболка piazza italia wr011446881,футболка,"(0, 8)",0,0,0,1,2,0,0,...,0,0,0,0,0,0,0,0,0,0
3,7) yi572-03x-one заколка для волос для девочки,заколка,"(17, 24)",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,одежда (вес) 1500,одежда,"(0, 6)",0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23407,"вода ""саирме"" с/г 500мл",вода,"(0, 4)",0,4,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23408,моя семя . 0.175л и ассортим,,"(0, 0)",1,4,0,0,1,2,0,...,0,0,0,0,0,0,0,0,0,0
23409,рулет бисквитн.яшкино клубничный со слив,рулет,"(0, 5)",0,4,0,1,2,0,0,...,0,0,0,0,0,0,0,0,0,0
23410,460075794371 почвогрунт цветочное счастье фас...,почвогрунт,"(13, 23)",0,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
oof["good_pred"] = probs2str(oof[list(range(66))].values, oof.name.tolist())
oof

,name,good,good_pos,strat,fold,0,1,2,3,4,...,57,58,59,60,61,62,63,64,65,good_pred
0,petmax бантик леопард с красн розой 2шт,бантик,"(7, 13)",0,0,0,0,0,1,2,...,0,0,0,0,0,0,0,0,0,бантик
1,87191 бусы для елки шарики_87191,бусы,"(6, 10)",0,0,0,0,0,1,2,...,0,0,0,0,0,0,0,0,0,бусы
2,футболка piazza italia wr011446881,футболка,"(0, 8)",0,0,0,1,2,0,0,...,0,0,0,0,0,0,0,0,0,футболка
3,7) yi572-03x-one заколка для волос для девочки,заколка,"(17, 24)",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,заколка
4,одежда (вес) 1500,одежда,"(0, 6)",0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,одежда
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23407,"вода ""саирме"" с/г 500мл",вода,"(0, 4)",0,4,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,вода
23408,моя семя . 0.175л и ассортим,,"(0, 0)",1,4,0,0,1,2,0,...,0,0,0,0,0,0,0,0,0,семя
23409,рулет бисквитн.яшкино клубничный со слив,рулет,"(0, 5)",0,4,0,1,2,0,0,...,0,0,0,0,0,0,0,0,0,рулет
23410,460075794371 почвогрунт цветочное счастье фас...,почвогрунт,"(13, 23)",0,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,почвогрунт


In [10]:
# id = 0
# print(oof[list(range(85))].iloc[id].tolist())
# print(oof[["name"]].iloc[id])

In [7]:
counter = 0
for id, row in oof[oof.good != oof.good_pred][["name", "good", "good_pred"]].iterrows():
    print(row["name"])
    print(f"true|{row.good}|")
    print(f"pred|{row.good_pred}|")
    print("#"*50)
    counter += 1

кашаб/п мистраль 40г*6 овсяная цельнозерновая манго-персик
true|каша|
pred||
##################################################
доска разделочная из кера белый/золотистый
true|доска|
pred|доска разделочная|
##################################################
гербер пюре 130г нежные овощи с кроликом гербер
true||
pred|пюре|
##################################################
бехеровка ( чехия) 50 1 порц.
true||
pred|бехеровка|
##################################################
jts433 стойка стабилизатора | перед прав/лев | trw
true|стойка|
pred|стойка стабилизатора|
##################################################
1/1 панкреатин табл п/о кишеч. 25 ед х60
true||
pred|панкреатин|
##################################################
карта acuvue 500р.
true||
pred|карта|
##################################################
подарок манго суш, 100 гр.
true|манго|
pred||
##################################################
aя автoмaтич ескaя  "kinex sl"   0.7 mm с4r-400 аpтикул 65926862674
true||
pr

In [12]:
counter

3508